In [82]:
import plotly.graph_objects as go
from dataTable import *
from weaponData import *
from dash import *

# 获取各种设置
settings = et.parse("settings.xml").getroot()
generalSettings = settings.find("general")
graphSettings = settings.find("graphSettings")
health = int(generalSettings.get("health"))
interpolation = generalSettings.get("interpolation")
xAxis = graphSettings.get("xAxis")
yAxis = graphSettings.get("yAxis")
# 所有trace的集合
traces = []
# 所有按钮的集合
buttons = []
def plotWeapon(weaponToPlot, hide = False):
    hoverFields = graphSettings.get("hoverFields").split(',')
    global fig
    id = 0
    try:
        id = int(weaponToPlot)
    except:
        # 把weaponToPlot从武器数据转换到ID
        id = getIDByName(weaponToPlot)
    weapon = WeaponData(id)
    killData = weapon.getKillData(health)
    x=killData[xAxis].tolist()
    y=killData[yAxis].tolist()
    # 获取悬浮菜单中的数据
    hoverText = []
    # 遍历X轴的所有数据
    for i in x:
        text = ''
        # 找到悬浮菜单中对应当前X的数值
        for j in hoverFields:
            text += j + ': ' + str(killData[killData[xAxis] == i][j].iloc[0]) + '<br>'
        hoverText.append(text)
    trace = go.Scatter(x=x, y=y, line_shape = interpolation, name = getTitleByWeapon(weapon),
                             hovertemplate = 
                                 '<br><b>' + yAxis + ': %{y}</b><br>' + 
                                 '<b>' + xAxis + ': %{x}</b><br>' +
                                 '%{text}', 
                                 text = hoverText,
                                 visible=not(hide)
                             )
    traces.append(trace)
    fig.add_trace(trace)
    # 更新格式被下移，暂时注释掉
    '''
    fig.update_layout(hovermode='x unified', 
    xaxis_title=xAxis, yaxis_title=yAxis, 
    width=float(generalSettings.get("sizeX")), height=float(generalSettings.get("sizeY")))
    '''
def getTitleByWeapon(weapon):
    return weapon.name + " (ID: {})".format(str(weapon.id))
def plotAll(hide = False):
    for i in weapons:
        plotWeapon(i.id, hide=hide)
def updateLayout():
    fig.update_layout(
    hovermode='x unified', 
    xaxis_title=xAxis, yaxis_title=yAxis, 
    width=float(generalSettings.get("sizeX")), height=float(generalSettings.get("sizeY")),
    updatemenus=[
        dict(
            type="dropdown",
            direction="down",
            buttons=buttons,
            pad={"r": 3, "t": 3},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top",
        )
    ]
    )

In [83]:
# 创建按钮列表，每个武器对应一个按钮
def getWeaponVisibility(weapon):
    for i in fig.data:
        if i.name == getTitleByWeapon(weapon):
            print(i.name + str(i.visible)) 
            return i.visible
def setWeaponVisibility(name, visible):
    weapon = getWeaponByName(name)
    for i in range(len(fig.data)):
        if fig.data[i].name == getTitleByWeapon(weapon):
            fig.data[i].visible = visible
def addDropdownMenu():
    global buttons
    buttons = []
    for i in weapons:
        # 定义按钮触发时的操作：仅显示当前武器
        button = dict(
            method="restyle",
            args=[{'visible':[i == x for x in weapons]}],
            #args=[{'visible':[i == j or getWeaponVisibility(j) for j in weapons]}], #遍历武器组中的所有数据，找到自身就把可见性打开
            args2=[{'visible':[i != x for x in weapons]}],
            label=i.name
        )
        buttons.append(button)

# 注意事项：
如果你对任何此文件之外的文件进行了修改（代码、数据、settings.xml设置文件等），记得在运行之前点Restart重启kernel，否则数据和代码都不会更新。
关于该程序的具体设置，请见README.MD
# IMPORTANT:
If you have modified any files outside of this file (code, data, settings.xml settings file, etc.), remember to restart the kernel by clicking on the Restart button before running, otherwise the data and code will not be updated.
For specific settings of this program, please refer to README-EN.MD

In [ ]:
fig = go.Figure()
# addDropdownMenu()
# 在此声明要绘制哪些武器，或者plotAll()绘制所有
'''
plotWeapon('M4')
plotWeapon('AK47')
plotWeapon('S1897')
plotWeapon('MP5')
plotWeapon('Baret')
'''
plotAll()
updateLayout()
app = Dash()

weaponNames = [i.name for i in weapons]
# App layout
app.layout = [
    html.Div(id='container'),
    dcc.Dropdown(weaponNames, 'Select Weapon', id='weaponsDropdown'),
    html.Button('Show All', id='showAll'),
    html.Button('Clear All', id='clearAll'),
    dcc.Graph(figure=fig)
]

@callback(
    Output('container', 'children'),
    Input('weaponsDropdown', 'value'),
    Input('showAll', 'n_clicks'), 
    Input('clearAll', 'n_clicks'), 
    prevent_initial_call=True
)
def OnInteract(dropDown, b1, b2):
    id = ctx.triggered_id
    if id == 'weaponsDropdown':
    # TODO 下拉菜单组件
        if (dropDown != None): 
            print(dropDown)
            setWeaponVisibility(dropDown, False)
    # TODO 显示所有按钮组件
    elif id == 'showAll':
        print(fig.data)
    # TODO 清除所有按钮组件
    elif id == 'clearAll':
        print('clearAll')
# Run the app
if __name__ == '__main__':
    app.run(debug=False)

M4
AK47
